# Project on Heart Disease (Group 73)

## Introduction

Heart disease is a broad term that refers to different heart conditions. Something in common is their effect on blood flow and dilation of veins to the heart. Other factors such as age, sex, and exercise are factors that are closely linked with heart disease. In our directory there are four data bases: Cleveland, Hungarian, Long Beach (California), and Switzerland. We decided to choose Long Beach due to similar culture (such as food and lifestyle) between Vancouver and Long beach. 


#test 1 with avery

#Prav Cells Be
low (helps out with reducing errors)

The relationship between these factors and their effect on predicting heart disease can be further examined through our research question: 
This project aims to provide a classification model which will predict whether a person has heart disease based on the factors of 

 1. Cleveland Clinic Foundation (cleveland.data)
     2. Hungarian Institute of Cardiology, Budapest (hungarian.data)
     3. V.A. Medical Center, Long Beach, CA (long-beach-va.data)
     4. University Hospital, Zurich, Switzerland (switzerland.data)


Cleveland Clinic Foundation (cleveland.data)
     2. Hungarian Institute of Cardiology, Budapest (hungarian.data)
     3. V.A. Medical Center, Long Beach, CA (long-beach-va.data)
     4. University Hospital, Zurich, Switzerland (switzerland.data)


This project aims to build a classification model to predict whether a patient presenting to the hospital would have a risk of heart disease by taking into account results from multiple medical tests. 
The dataset (https://archive.ics.uci.edu/ml/datasets/Heart+Disease) provides us with multiple attributes such as age and sex as well as a categorical attribute of whether a patient is in a risk of having a heart attack

The term “heart disease” refers to several types of heart conditions. The most common type of heart disease in the United States is coronary artery disease (CAD), which affects the blood flow to the heart. Decreased blood flow can cause a heart attack.


## Data Preparation

In [1]:
#Adding in librarys
library(tidyverse)
library(tidymodels)


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [2]:
#loading dataset from database

# url <- "https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset/download?datasetVersionNumber=2"
# longbeach_data <- read_csv(url, col_names = TRUE)
# longbeach_data
longbeach_data <- read_csv("data/bad_data/heart.csv")
longbeach_data


Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (14): age, sex, cp, trtbps, chol, fbs, restecg, thalachh, exng, oldpeak,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
52,1,2,172,199,1,1,162,0,0.5,2,0,3,1


In [3]:
#renaming data frame's column names accordingly
# longbeach_set <- rename(longbeach_data,
#     age = X1, 
#     sex = X2, 
#     cp = X3,
#     trestbps = X4, 
#     chol = X5, 
#     fbs = X6,
#     restecg = X7,
#     thalach = X8, 
#     exang = X9, 
#     oldpeak = X10,                        
#     slope = X11, 
#     ca = X12,
#     thal = X13, 
#     num = X14)

In [8]:
#taking our most important columns
#then changing num to factor
longbeach_select <- longbeach_data |>
    select(age, sex, trtbps, chol, oldpeak, output) |>
mutate(output = as.factor(output))

# |>
#    mutate(num = as.factor(num))

# longbeach_select1 <- longbeach_select |>
# filter(age != "?") |>
# filter(chol != "?") |>
# filter(thalach != "?") |>
# filter(trestbps != "?") |>
# filter(oldpeak != "?") |>
# filter(num != "?")
#longbeach_select1

# longbeach_select_final <- longbeach_select1 |>
# mutate(chol = as.double(chol)) |>
# mutate(thalach = as.double(thalach)) |>
# mutate(trestbps = as.double(trestbps)) |>
# mutate(oldpeak = as.double(oldpeak))

# longbeach_select_final |>
# print(n=10)




longbeach_select

age,sex,trtbps,chol,oldpeak,output
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
63,1,145,233,2.3,1
37,1,130,250,3.5,1
41,0,130,204,1.4,1
56,1,120,236,0.8,1
57,0,120,354,0.6,1
57,1,140,192,0.4,1
56,0,140,294,1.3,1
44,1,120,263,0.0,1
52,1,172,199,0.5,1


To clean up the data, we first renamed all the columns accordingly to all the given attributes of the dataset. Then we took out all the unused attributes and changed variable num to factor because it is a categorical statical variable.

In [10]:
# split data into training set and testing set


longbeach_split <- initial_split(longbeach_select, prop=0.75, strata=output)
longbeach_train <- training(longbeach_split)
longbeach_test <- testing(longbeach_split)

We then split the cleaned data into training set and testing set, where 75% of the data are used to training while the remaining 25% will be used for testing. The proportion are chosen so that there is sufficient data to train the model with sufficient data to test the model and obtain the accuracy of the model. 

## Method

We will build the classification model using k-nearest neighbors. We will build it using the training data type longbeach_train and test it for accuracy using the testing data type longbeach_test. The number of k we use will be tuned to have the highest accuracy through cross-validation. The predictor variables we use will only be variables that have double as their data type and not factor data type. This is because factor data types can not be used for k-nearest neighbors as distance can not be calculated for factors. 

We can visualize the tuning using cross-validation by plotting a graph with number of k vs accuracy.


## Expected Outcomes and Significance

From this research project, we expect to find a correlation between risk factors (specifically, a person's age, cholesterol, resting heart rate, resting blood pressure, and ST depression) and the overall likelyhood that they have heart disease. With this research, we hope to create a model that can predict (to a high level of accuracy) whether a person has heart disease. We will use data taken from the UCI Machine Learning Repository's Heart Disease Dataset to train our model to predict whether someone has heart disease based on the 5 risk factors listed earlier. If our model is successful, it will be a great aid to the medical community by creating fast and easy diagnosis of heart disease in patients, and will allow doctors to quickly assist those in need. A doctor could simply input the measurements taken for each of the 5 risk factors and the model will determine whether the patient is developing heart disease or not. 

After our model is complete, future questions may be raised on this topic such as:
- Are there other diseases that can be accurately predicted based on a chosen set of measurable risk factors?
- Will it ever be ethical to nearly remove a doctor from the situation and trust a model instead?
- Is it possible to create a model that is 100% accurate?
- How might a model, such as ours, affect the speed at which heart disease is detected and treated, and could this lead to more lives saved in the future?

In [13]:
#YAY we got our final data
#now we need to split into training and testing

#longbeach_train
#longbeach_test

long_beach_recipe <- recipe(output ~ age + sex + trtbps + chol + oldpeak, data = longbeach_select) |>
   step_scale(all_predictors()) |>
   step_center(all_predictors())

long_beach_recipe

Recipe

Inputs:

      role #variables
   outcome          1
 predictor          5

Operations:

Scaling for all_predictors()
Centering for all_predictors()

In [63]:
# Now we need to make KNN spec and workflow

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 91) |>
      set_engine("kknn") |>
      set_mode("classification")
#knn_spec
long_beach_fit <- workflow() |>
      add_recipe(long_beach_recipe) |>
      add_model(knn_spec) |>
      fit(data = longbeach_train)
#long_beach_fit

In [64]:
long_beach_predictions <- predict(long_beach_fit , longbeach_test) |>
      bind_cols(longbeach_test)
long_beach_predictions |>
print(n=10)

# A tibble: 77 × 7
   .pred_class   age   sex trtbps  chol oldpeak output
   <fct>       <dbl> <dbl>  <dbl> <dbl>   <dbl> <fct> 
 1 0              37     1    130   250     3.5 1     
 2 1              57     1    140   192     0.4 1     
 3 1              56     0    140   294     1.3 1     
 4 1              44     1    120   263     0   1     
 5 1              52     1    172   199     0.5 1     
 6 1              49     1    130   266     0.6 1     
 7 0              64     1    110   211     1.8 1     
 8 1              58     0    120   340     0   1     
 9 1              43     1    150   247     1.5 1     
10 1              42     1    140   226     0   1     
# … with 67 more rows


In [65]:
#Now we need to find accuracy

long_beach__prediction_accuracy <- long_beach_predictions |> #df
        metrics(truth = output, estimate = .pred_class)   #real vs estimations

long_beach__prediction_accuracy



.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.7532468
kap,binary,0.4963855
